In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['patch.force_edgecolor'] = True
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [61]:
#Gives heatmap of null values
def get_heatmap(df):
    #This function gives heatmap of all NaN values
    plt.figure(figsize=(10,6))
    sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='viridis')
    plt.tight_layout()
    return plt.show()

In [62]:
filepath = 'https://raw.githubusercontent.com/jubins/ML-TwitterBotDetection/master/FinalCode/kaggle_data/'

train_df = pd.read_csv(filepath+'training_data_2_csv_UTF.csv')
test_df_backup = pd.read_csv(filepath+'test_data_4_students.csv', sep='\t')
test_df_backup['id'] = test_df_backup.id.apply(lambda x: int(x))

In [63]:
test_df = test_df_backup
train_bots = train_df[train_df.bot==1]
train_nonbots = train_df[train_df.bot==0]

train_bots['screen_name_binary'] = (train_bots.screen_name.str.contains("bot", case=False)==True)
train_nonbots['screen_name_binary'] = (train_nonbots.screen_name.str.contains("bot", case=False))

training_df = pd.concat([train_bots, train_nonbots])

test_df['screen_name_binary'] = (test_df.screen_name.str.contains("bot", case=False)==True)

In [64]:
#test_df[(test_df.screen_name.str.contains("bot", case=False)==True)] #206 these all are bots
listed_count_df = test_df[(test_df.screen_name.str.contains("bot", case=False)==False)]

final_df = test_df[(test_df.screen_name.str.contains("bot", case=False)==True)] #206 these all are bots
final_df.bot=1
final_df = final_df[['id', 'bot']]
final_df.shape[0]

206

In [65]:
listed_count_df.listed_count = listed_count_df.listed_count.apply(lambda x: 0 if x=='None' else x)
listed_count_df.listed_count = listed_count_df.listed_count.apply(lambda x: int(x))

In [66]:
listed_count_df[listed_count_df.listed_count>16000].shape #these all are nonbots
final_df1 = listed_count_df[listed_count_df.listed_count>16000][['id', 'bot']]
final_df1.bot=0
final_df1.shape
final_df = pd.concat([final_df, final_df1])
final_df.shape[0]

300

In [67]:
verified_df = listed_count_df[listed_count_df.listed_count<16000]

In [68]:
verified_df[verified_df.verified=='TRUE'].shape #these all are nonbots
final_df1 = verified_df[verified_df.verified=='TRUE'][['id', 'bot']]
final_df1.bot=0
final_df = pd.concat([final_df, final_df1])
final_df.shape

(408, 2)

In [69]:
description_df = verified_df[verified_df.verified!='TRUE']

In [70]:
description_df[description_df.description.str.contains("bot", case=False, na=False)].shape #these all are bots

final_df1 = description_df[description_df.description.str.contains("bot", case=False, na=False)][['id', 'bot']]
final_df1.bot=1
final_df = pd.concat([final_df, final_df1])
final_df.shape

(454, 2)

In [32]:
status_df = description_df[description_df.description.str.contains("bot", case=False, na=False)==False]

In [33]:
status_df[status_df.status.str.contains("bot", case=False, na=False)].shape #these all are bots
final_df1 = status_df[status_df.status.str.contains("bot", case=False, na=False)][['id', 'bot']]
final_df1.bot=1
final_df = pd.concat([final_df, final_df1])
final_df.shape

(470, 2)

In [34]:
buzzfeed_df = status_df[status_df.status.str.contains("bot", case=False, na=False)==False]

buzzfeed_df[buzzfeed_df.description.str.contains("buzzfeed", case=False, na=False)].shape #these all are nonbots

final_df1 = buzzfeed_df[buzzfeed_df.description.str.contains("buzzfeed", case=False, na=False)][['id','bot']]
final_df1.bot=0
final_df = pd.concat([final_df, final_df1])
final_df.shape

(486, 2)

In [35]:
name_df = buzzfeed_df[buzzfeed_df.description.str.contains("buzzfeed", case=False, na=False)==False]
condition = (name_df.name.str.contains("bot", case=False, na=False)) | (name_df.screen_name.str.contains("b0t", case=False, na=False))
name_df[condition].shape #these all are bots

final_df1 = name_df[condition][['id','bot']]
final_df1.bot=1
final_df = pd.concat([final_df, final_df1])
final_df.shape

(492, 2)

In [43]:
truth_df = name_df[(name_df.name.str.contains("bot", case=False, na=False))==False] #shape: 83

condition = ((truth_df.screen_name.str.contains("truth", case=False, na=False)) | 
            (truth_df.name.str.contains("truth", case=False, na=False)) | 
            (truth_df.screen_name.str.contains("anony", case=False, na=False)))

truth_df[condition].shape #these all are bots

final_df1 = truth_df[condition][['id','bot']]
final_df1.bot=1
final_df = pd.concat([final_df, final_df1])
final_df.shape

(504, 2)

In [44]:
statuses_count_df = truth_df[~condition]
condition = ((statuses_count_df.description.str.contains('cannabis', case=False, na=False)) | (statuses_count_df.description.str.contains('mishear', case=False, na=False))) #these all are bots
statuses_count_df[condition]

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,...,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot,screen_name_binary
16,42382447,42382447,Agostinozoida,Agostinozoida,Agostinozoida,https://t.co/2gdWyQjNq4,964,331,15,5/25/2009 10:02,...,FALSE,6389,en,"{'place': None, 'retweeted_status': {'place': ...",FALSE,FALSE,TRUE,laughkommander,NaN,False
17,43993280,43993280,AhmedBharoocha,NaN,I forget to tweet,https://t.co/iM3lj0uBHg,5254,2407,102,6/1/2009 22:44,...,FALSE,3392,en,"{'place': None, 'retweeted_status': {'place': ...",TRUE,FALSE,TRUE,Ahmed Bharoocha,NaN,False
45,401790812,401790812,AveryFunny,Los Angeles,Actor Comedian Keys. Funny Or Die. Comedy Cent...,https://t.co/2KY9WXbjaI,2446,1823,28,10/31/2011 2:52,...,FALSE,2386,en,"{'place': None, 'retweeted_status': {'place': ...",FALSE,FALSE,FALSE,Avery Pearson,NaN,False
56,827983208,827983208,benkreimer,United States,I'm a free-range journalism technologist worki...,http://t.co/4575W1RLCd,1116,422,101,Mon Sep 17 00:01:57 +0000 2012,...,FALSE,261,en,{'created_at': 'Fri Apr 07 21:12:12 +0000 2017...,TRUE,FALSE,FALSE,Ben Kreimer,NaN,False
57,72878717,72878717,bglading,"Nashville, TN","Founder-Candid Career, LLC",http://t.co/ofQIjeU4TI,82,77,0,9/9/2009 15:32,...,FALSE,5,en,"{'place': None, 'retweeted': False, 'favorited...",TRUE,FALSE,FALSE,billy glading,NaN,False
59,43091065,43091065,bigangcomic,Los Angeles,Award Winning Comedian and Actor for inquiries...,https://t.co/bCxywQFuwr,5782,6331,123,5/28/2009 11:25,...,FALSE,4901,en,"{'place': None, 'retweeted': False, 'favorited...",FALSE,FALSE,FALSE,Angelo Tsarouchas,NaN,False
65,783048890,783048890,bnels13,,be kind | Insta: bnels_,https://t.co/bpIMp2bLwF,1250,925,2,8/26/2012 19:29,...,FALSE,4970,en,"{'place': None, 'retweeted': False, 'favorited...",FALSE,FALSE,TRUE,bailee,NaN,False
66,274656070,274656070,bobzany,Comedy Club near you Bay-bee!,What hasn't he done...,http://t.co/8nrXLhd6aK,21865,701,173,3/30/2011 19:34,...,FALSE,6518,en,"{'place': None, 'retweeted_status': {'place': ...",TRUE,FALSE,TRUE,Bob Zany,NaN,False
80,26436076,26436076,Bradymatthews13,"iPhone: 32.762235,-117.213986",My arms look fat.,NaN,1507,557,32,3/25/2009 5:10,...,FALSE,5925,en,"{'place': {'attributes': {}, 'country_code': '...",FALSE,FALSE,FALSE,Brady Matthews,NaN,False
85,165500655,165500655,brianwohl,Los Angeles,I like Fun. Let's make Fun. - Octavarius/Wrest...,https://t.co/JzR6lAEunJ,951,1110,18,7/11/2010 19:12,...,FALSE,3303,en,"{'place': None, 'retweeted': False, 'favorited...",TRUE,FALSE,FALSE,Brian Wohl,NaN,False


In [208]:
statuses_count_df[~condition].shape

(77, 21)

In [209]:
final_df1 = statuses_count_df[~condition][['id','bot']] #these all are nonbots
final_df1.bot=0
final_df = pd.concat([final_df, final_df1])
final_df.shape

(575, 2)

In [210]:
#preparing subission file
final_df.to_csv('submission.csv', index=False)

#2381315456, 159966364